In [ ]:
!pip install fairlearn

In [ ]:
# Example
if __name__ == "__main__":
    df = pd.read_csv('data/processed/cleaned_jobs_tech_aug.csv')
    # Assume 'recommended' column (1 if in top-10, 0 else) from recs
    df['recommended'] = np.random.randint(0, 2, len(df))  # Synthetic for demo
    audit = bias_audit(df)
    print(audit)
    # Debias example (use your ranking NN)
    features = np.random.rand(len(df), 5)  # E.g., embedding sim + features
    labels = df['recommended']
    sensitive = df['job_level']
    mitigator = debias_model(RandomForestClassifier(), features, labels, sensitive)  # Example model

In [ ]:
import joblib

joblib.dump(mitigator, 'model/debiased_model.pkl')

In [ ]:
!pip install shap

In [ ]:
import shap
from sklearn.ensemble import RandomForestRegressor  # Proxy for explainer

def explain_recs(model, features):
    explainer = shap.Explainer(model)
    shap_values = explainer(features)
    shap.summary_plot(shap_values, features, feature_names=['sim_score', 'skill_match', 'location_match'])
    return shap_values

# Example
if __name__ == "__main__":
    # Proxy model (use your ranking NN or RF for demo).
    proxy_model = RandomForestRegressor().fit(features, labels)
    shap_values = explain_recs(proxy_model, features)
    # For output: "Matches 80% on skills like PyTorch (SHAP +0.5)"

In [ ]:
# notebooks/advanced_eval.ipynb
import pandas as pd
import numpy as np
import faiss
#from src.embedding_model import generate_embeddings, recommend_jobs  # Import from Step 3
#from src.advanced_features import bias_audit, explain_recs  # From Step 4
from sklearn.ensemble import RandomForestRegressor  # For proxy if no NN

# Load data (adjust paths)
df = pd.read_csv('data/processed/cleaned_jobs_tech_aug.csv')
job_texts = df['cleaned_summary'].tolist()
job_embeddings = np.load('data/processed/job_embeddings.npy')
index = faiss.read_index('data/processed/faiss_index.index')

# Example resume text
resume_text = "master computer science ml dl pytorch nlp bert hugging face transformers scikit-learn faiss ethical ai bias mitigation fairlearn streamlit"

# Get recommendations
recs = recommend_jobs(resume_text, job_texts, df, top_k=10)

# Bias audit (add 'recommended' if missing)
recs['recommended'] = 1
bias_audit(recs)

# Explainability (derive features from recs; e.g., sim_score from column, dummy skill/location)
rec_features = np.column_stack((recs['similarity_score'], np.random.rand(len(recs), 2)))  # Real: [sim_score, skill_match, location_match]

# Define or load model (use proxy RF if no Step 3 NN; or load torch model)
proxy_model = RandomForestRegressor().fit(rec_features, recs['recommended'])  # Train proxy on rec data
explain_recs(proxy_model, rec_features)  # Use proxy_model instead of 'your_model'